In [1]:
from vedicastro.VedicAstro import VedicHoroscopeData
from vedicastro.utils import pretty_data_table
from flatlib import const
from pprint import pprint
import pandas as pd
import numpy as np

In [2]:
def get_sign_name(degree):
    index = int(degree // 30)
    return ZODIAC_SIGNS[index % 12]

def format_longitude(degree):
    sign_index = int(degree // 30)
    sign_deg = degree % 30
    minutes = (sign_deg - int(sign_deg)) * 60
    return f"{ZODIAC_SIGNS[sign_index]} {int(sign_deg)}° {int(minutes)}′"
    
def get_coordinates_by_geo(city, country):
    geolocator = Nominatim(user_agent="vedic_chart_app")
    location = geolocator.geocode(f"{city}, {country}")
    if location:
        return location.latitude, location.longitude
    else:
        raise ValueError("Местоположение не найдено")

def get_birth_timezone(birth_datetime_str, latitude, longitude):
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=latitude, lng=longitude)
    if not tz_name:
        raise ValueError("Не удалось определить часовой пояс для координат")
    return tz_name

def get_utc_birth_datetime(birth_datetime_str, tz_name):
    tz = ZoneInfo(tz_name)
    dt_local = datetime.strptime(birth_datetime_str, "%d.%m.%Y %H:%M").replace(tzinfo=tz)
    return dt_local.astimezone(ZoneInfo("UTC"))

def convert_utc_to_local_string(dt_utc, tz_name):
    local_dt = dt_utc.astimezone(ZoneInfo(tz_name))
    time_str = local_dt.strftime("%H:%M %d/%m/%Y %z")  # без двоеточия в смещении
    # если хочешь формат именно "+05:00" — нужно вручную отформатировать
    offset = local_dt.strftime("%z")  # пример: +0500
    offset_formatted = f"{offset[:3]}:{offset[3:]}"  # → +05:00
    final_str = f"{local_dt.strftime('%H:%M %d/%m/%Y')} {offset_formatted}"
    return final_str

def translate_chart_to_russian(df):
    planet_map = {
        "Sun": "Солнце", "Moon": "Луна", "Mars": "Марс", "Mercury": "Меркурий", "Jupiter": "Юпитер",
        "Venus": "Венера", "Saturn": "Сатурн", "Rahu": "Раху", "Ketu": "Кету",
        "Uranus": "Уран", "Neptune": "Нептун", "Pluto": "Плутон", "Chiron": "Хирон",
        "Asc": "Асцендент", "Syzygy": "Сизигия"
    }

    sign_map = {
        "Aries": "Овен", "Taurus": "Телец", "Gemini": "Близнецы", "Cancer": "Рак",
        "Leo": "Лев", "Virgo": "Дева", "Libra": "Весы", "Scorpio": "Скорпион",
        "Sagittarius": "Стрелец", "Capricorn": "Козерог", "Aquarius": "Водолей", "Pisces": "Рыбы"
    }

    nakshatra_map = {
        "Ashwini": "Ашвини", "Bharani": "Бхарани", "Krittika": "Криттика", "Rohini": "Рохини",
        "Mrigashīrsha": "Мригашира", "Ardra": "Ардра", "Punarvasu": "Пунарвасу",
        "Pushya": "Пушья", "Ashlesha": "Ашлеша", "Magha": "Магха", "PurvaPhalgunī": "Пурва Пхалгуни",
        "UttaraPhalgunī": "Уттара Пхалгуни", "Hasta": "Хаста", "Chitra": "Читра",
        "Svati": "Свати", "Vishakha": "Вишакха", "Anuradha": "Анурадха", "Jyeshtha": "Джьештха",
        "Mula": "Мула", "PurvaAshadha": "Пурва Ашадха", "UttaraAshadha": "Уттара Ашадха",
        "Shravana": "Шравана", "Dhanishta": "Дхаништха", "Shatabhishak": "Шатабхиша",
        "PurvaBhādrapadā": "Пурва Бхадрапада", "UttaraBhādrapadā": "Уттара Бхадрапада",
        "Revati": "Ревати"
    }

    df_translated = df.copy()

    for col, mapping in {
        "Планета": planet_map,
        "Знак": sign_map,
        "Накшатра": nakshatra_map,
        "Управитель": planet_map,
        "Управитель накшатры": planet_map
    }.items():
        if col in df_translated.columns:
            df_translated[col] = df_translated[col].map(mapping).fillna(df_translated[col])

    return df_translated

# Обновлённая версия функции с улучшениями: формат Асцендента + столбец "Управляет"
def extend_planet_df(df):

    # Добавим столбец "Управитель"
    df["Управитель"] = df["Знак"].map(SIGN_RULERS)

    df["Дом"] = df["Знак"].apply(get_house)

    # Добавим столбец "Управляет" — для каждой планеты: какими домами она управляет
    planet_to_signs = {}
    for sign, ruler in SIGN_RULERS.items():
        planet_to_signs.setdefault(ruler, []).append(sign)

    df["Управляет домами"] = df["Планета"].apply(get_house_ruled)

    return df

# Добавим столбец "Дом"
def get_house(sign, asc_sign_index):
    sign_index = list(SIGN_RULERS.keys()).index(sign)
    return (sign_index - asc_sign_index) % 12 + 1

def get_house_ruled(planet, asc_sign_index):
    signs_ruled = planet_to_signs.get(planet, [])
    houses = sorted([(list(SIGN_RULERS.keys()).index(sign) - asc_sign_index) % 12 + 1 for sign in signs_ruled])
    return ", ".join(str(h) for h in houses)

def build_navamsa(df):
    navamsa_signs = []

    for i, row in df.iterrows():
        rashi_sign = row["Знак"]
        degree_in_sign = row["Градус в знаке"]
        planet = row["Планета"]

        if rashi_sign is None or degree_in_sign is None:
            navamsa_signs.append(None)
            continue

        part = int(degree_in_sign // (30 / 9))  # каждая навамша = 3°20′
        d9_sign = D1_TO_D9_MAP[rashi_sign][part]

        # Отладочный вывод для Асцендента
        if planet == "Асцендент":
            print(f"[DEBUG] Асцендент: знак={rashi_sign}, градус={degree_in_sign:.4f}, часть={part}, Навамша={d9_sign}")

        navamsa_signs.append(d9_sign)

    df["Навамша (знак)"] = navamsa_signs
    return df

def filter_main_grahas(df):
    main_grahas = [
        "Асцендент", "Солнце", "Луна", "Марс", "Меркурий",
        "Юпитер", "Венера", "Сатурн", "Раху", "Кету"
    ]
    return df[df["Планета"].isin(main_grahas)].reset_index(drop=True)

def add_house_info(df):

    # Порядок знаков в сидерическом зодиаке
    rasi_order = [
        "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева",
        "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"
    ]

    # Управители знаков (по ведической традиции)
    sign_lords = {
        "Овен": "Марс",       "Телец": "Венера",    "Близнецы": "Меркурий",
        "Рак": "Луна",        "Лев": "Солнце",      "Дева": "Меркурий",
        "Весы": "Венера",     "Скорпион": "Марс",   "Стрелец": "Юпитер",
        "Козерог": "Сатурн",  "Водолей": "Сатурн",  "Рыбы": "Юпитер"
    }

    # 1. Определяем знак лагны
    asc_row = df[df["Планета"] == "Асцендент"]
    if asc_row.empty:
        raise ValueError("В DataFrame нет строки с Асцендентом")
    asc_sign = asc_row.iloc[0]["Знак"]

    # 2. Домовая последовательность
    rasi_from_asc = rasi_order[rasi_order.index(asc_sign):] + rasi_order[:rasi_order.index(asc_sign)]
    sign_to_house = {sign: i + 1 for i, sign in enumerate(rasi_from_asc)}

    # 3. Для каждой планеты: дом пребывания и дома управления
    def planet_house(zodiac_sign):
        return sign_to_house.get(zodiac_sign, np.nan)

    def planet_rules(planet):
        ruled_signs = [sign for sign, lord in sign_lords.items() if lord == planet]
        ruled_houses = [sign_to_house[sign] for sign in ruled_signs]
        return ", ".join(str(h) for h in sorted(ruled_houses))

    df = df.copy()
    df["Дом"] = df["Знак"].apply(planet_house)
    df["Управляет домами"] = df["Планета"].apply(planet_rules)
    #df = df[df["Планета"] != "Асцендент"].reset_index(drop=True)

    return df

In [3]:
ZODIAC_SIGNS = [
    "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева",
    "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"
]

sign_rus = {
    "Aries": "Овен", "Taurus": "Телец", "Gemini": "Близнецы", "Cancer": "Рак",
    "Leo": "Лев", "Virgo": "Дева", "Libra": "Весы", "Scorpio": "Скорпион",
    "Sagittarius": "Стрелец", "Capricorn": "Козерог", "Aquarius": "Водолей", "Pisces": "Рыбы"
}

SIGN_RULERS = {
    "Овен": "Марс",
    "Телец": "Венера",
    "Близнецы": "Меркурий",
    "Рак": "Луна",
    "Лев": "Солнце",
    "Дева": "Меркурий",
    "Весы": "Венера",
    "Скорпион": "Марс",
    "Стрелец": "Юпитер",
    "Козерог": "Сатурн",
    "Водолей": "Сатурн",
    "Рыбы": "Юпитер",
}

D1_TO_D9_MAP = {
    "Овен":      ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Телец":     ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Близнецы":  ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рак":       ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Лев":       ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Дева":      ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Весы":      ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Скорпион":  ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
    "Стрелец":   ["Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец"],
    "Козерог":   ["Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы", "Рак", "Лев", "Дева"],
    "Водолей":   ["Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы", "Овен", "Телец", "Близнецы"],
    "Рыбы":      ["Рак", "Лев", "Дева", "Весы", "Скорпион", "Стрелец", "Козерог", "Водолей", "Рыбы"],
}

planetName_eng_to_ru = {
    "Sun": "Солнце",
    "Moon": "Луна",
    "Mars": "Марс",
    "Mercury": "Меркурий",
    "Jupiter": "Юпитер",
    "Venus": "Венера",
    "Saturn": "Сатурн",
    "Rahu": "Раху",
    "Ketu": "Кету"
}

columns_mapping = {
    'Object': 'Планета',
    'Rasi': 'Знак',
    'isRetroGrade': 'Ретроградность',
    'SignLonDMS': 'Градусы',
    'Nakshatra': 'Накшатра',
    'RasiLord': 'Управитель',
    'NakshatraLord': 'Управитель накшатры'
}

In [4]:
year = 1987
month = 1
day = 20
hour = 10
minute = 53
secs = 0
latitude, longitude, utc = 56.05, 60.44, "+5:00" ## New York
ayan = "Lahiri"
house_system = "Placidus" # Default House system for Krishnamurti Paddhati system

In [5]:
vhd = VedicHoroscopeData(year = year, month = month, day = day, hour = hour, minute = minute, second = secs, utc = utc, 
                        latitude = latitude, longitude = longitude, ayanamsa = ayan, house_system = house_system)

In [6]:
vhd.get_ayanamsa()
chart = vhd.generate_chart()
planets_data = vhd.get_planets_data_from_chart(chart)
planets_df = pd.DataFrame(planets_data)
# planets_df.select(["Object", "LonDecDeg", "Nakshatra", "RasiLord", "NakshatraLord", "SubLord"])
planets_df = planets_df.drop(columns=['LonDecDeg', 'SignLonDecDeg', 'LatDMS', 'SubLord', 'SubSubLord', 'HouseNr'])
planets_df = planets_df.rename(columns=columns_mapping)
planets_df = translate_chart_to_russian(planets_df)

In [7]:
planets_df = filter_main_grahas(planets_df)

In [8]:
planets_df

,Планета,Знак,Ретроградность,Градусы,Накшатра,Управитель,Управитель накшатры
0,Асцендент,Водолей,None,+29:31:16,Пурва Бхадрапада,Сатурн,Юпитер
1,Солнце,Козерог,False,+05:57:07,Уттара Ашадха,Сатурн,Солнце
2,Луна,Дева,False,+03:32:51,Уттара Пхалгуни,Меркурий,Солнце
3,Меркурий,Козерог,False,+10:53:37,Шравана,Сатурн,Луна
4,Венера,Скорпион,False,+19:09:01,Джьештха,Марс,Меркурий
5,Марс,Рыбы,False,+14:31:23,Уттара Бхадрапада,Юпитер,Сатурн
6,Юпитер,Водолей,False,+27:14:06,Пурва Бхадрапада,Сатурн,Юпитер
7,Сатурн,Скорпион,False,+23:43:54,Джьештха,Марс,Меркурий
8,Раху,Рыбы,True,+21:48:01,Ревати,Юпитер,Меркурий
9,Кету,Дева,True,+21:48:01,Хаста,Меркурий,Луна


In [9]:
final_chart = add_house_info(planets_df)

In [10]:
final_chart

,Планета,Знак,Ретроградность,Градусы,Накшатра,Управитель,Управитель накшатры,Дом,Управляет домами
0,Асцендент,Водолей,None,+29:31:16,Пурва Бхадрапада,Сатурн,Юпитер,1,
1,Солнце,Козерог,False,+05:57:07,Уттара Ашадха,Сатурн,Солнце,12,7
2,Луна,Дева,False,+03:32:51,Уттара Пхалгуни,Меркурий,Солнце,8,6
3,Меркурий,Козерог,False,+10:53:37,Шравана,Сатурн,Луна,12,"5, 8"
4,Венера,Скорпион,False,+19:09:01,Джьештха,Марс,Меркурий,10,"4, 9"
5,Марс,Рыбы,False,+14:31:23,Уттара Бхадрапада,Юпитер,Сатурн,2,"3, 10"
6,Юпитер,Водолей,False,+27:14:06,Пурва Бхадрапада,Сатурн,Юпитер,1,"2, 11"
7,Сатурн,Скорпион,False,+23:43:54,Джьештха,Марс,Меркурий,10,"1, 12"
8,Раху,Рыбы,True,+21:48:01,Ревати,Юпитер,Меркурий,2,
9,Кету,Дева,True,+21:48:01,Хаста,Меркурий,Луна,8,


In [13]:
#vd = vhd.compute_vimshottari_dasa(chart)
vd